
このノートブックを実行するには、次の追加ライブラリが必要です。 Colab での実行は実験的なものであることに注意してください。問題がある場合は、Github の問題を報告してください。


In [ ]:
!pip install d2l==1.0.0-beta0



# 自然言語推論とデータセット

:label: `sec_natural-language-inference-and-dataset`

 :numref: `sec_sentiment`では、感情分析の問題について説明しました。このタスクの目的は、単一のテキスト シーケンスを一連の感情の極性など、事前定義されたカテゴリに分類することです。ただし、ある文が別の文から推論できるかどうかを判断したり、意味的に同等の文を識別して冗長性を排除したりする必要がある場合、1 つのテキスト シーケンスを分類する方法を知っているだけでは不十分です。代わりに、テキスト シーケンスのペアを推論できる必要があります。

## 自然言語推論

*自然言語推論で*は、両方がテキスト シーケンスである*前提*から*仮説*を推論できるかどうかを研究します。言い換えれば、自然言語推論は、テキスト シーケンスのペア間の論理関係を決定します。このような関係は通常、次の 3 つのタイプに分類されます。
- *含意*: 仮説は前提から推測できます。
- *矛盾*: 仮説の否定は前提から推測できます。
- *中立*: その他すべての場合。

自然言語推論は、テキスト含意の認識タスクとしても知られています。たとえば、次のペアは*含意*としてラベル付けされます。これは、仮説の「愛情を示す」が前提の「抱き合う」から推測できるためです。

> 前提: 二人の女性が抱き合っています。

> 仮説: 2 人の女性が愛情を示しています。

次の例は、「コーディング例を実行している」ということが「スリープ中」ではなく「スリープしていない」ことを示しているため、*矛盾*しています。

> 前提: 男性が Dive into Deep Learning のコーディング例を実行しています。

> 仮説: その男は眠っています。

 3 番目の例は、「私たちのためにパフォーマンスを行っている」という事実から「有名」も「有名ではない」も推測できないため、*中立*関係を示しています。

> 前提: ミュージシャンは私たちのために演奏しています。

> 仮説: そのミュージシャンは有名です。

自然言語推論は、自然言語を理解するための中心的なトピックです。情報検索からオープンドメインの質問応答まで、幅広い用途に利用されています。この問題を研究するには、まず、一般的な自然言語推論ベンチマーク データセットを調査します。

## スタンフォード自然言語推論 (SNLI) データセット

[ **Stanford Natural Language Inference (SNLI) Corpus** ] は、500,000 を超えるラベル付きの英語文ペアのコレクションです (引用: `Bowman.Angeli.Potts.ea.2015` )。抽出した SNLI データセットをダウンロードしてパス`../data/snli_1.0`に保存します。


In [1]:
import os
import re
import torch
from torch import nn
from d2l import torch as d2l

#@save
d2l.DATA_HUB['SNLI'] = (
    'https://nlp.stanford.edu/projects/snli/snli_1.0.zip',
    '9fcde07509c7e87ec61c640c1b2753d9041758e4')

data_dir = d2l.download_extract('SNLI')


### [**データセットの読み取り**]

元の SNLI データセットには、実験で実際に必要な情報よりもはるかに豊富な情報が含まれています。したがって、データセットの一部のみを抽出し、前提、仮説、およびそれらのラベルのリストを返す関数`read_snli`を定義します。


In [2]:
#@save
def read_snli(data_dir, is_train):
    """Read the SNLI dataset into premises, hypotheses, and labels."""
    def extract_text(s):
        # Remove information that will not be used by us
        s = re.sub('\\(', '', s)
        s = re.sub('\\)', '', s)
        # Substitute two or more consecutive whitespace with space
        s = re.sub('\\s{2,}', ' ', s)
        return s.strip()
    label_set = {'entailment': 0, 'contradiction': 1, 'neutral': 2}
    file_name = os.path.join(data_dir, 'snli_1.0_train.txt'
                             if is_train else 'snli_1.0_test.txt')
    with open(file_name, 'r') as f:
        rows = [row.split('\t') for row in f.readlines()[1:]]
    premises = [extract_text(row[1]) for row in rows if row[0] in label_set]
    hypotheses = [extract_text(row[2]) for row in rows if row[0] in label_set]
    labels = [label_set[row[0]] for row in rows if row[0] in label_set]
    return premises, hypotheses, labels


ここで、前提と仮説、およびそれらのラベル (「0」、「1」、「2」はそれぞれ「含意」、「矛盾」、「中立」に対応します) の [**最初の 3 つのペアを出力**] しましょう。


In [3]:
train_data = read_snli(data_dir, is_train=True)
for x0, x1, y in zip(train_data[0][:3], train_data[1][:3], train_data[2][:3]):
    print('premise:', x0)
    print('hypothesis:', x1)
    print('label:', y)

premise: A person on a horse jumps over a broken down airplane .
hypothesis: A person is training his horse for a competition .
label: 2
premise: A person on a horse jumps over a broken down airplane .
hypothesis: A person is at a diner , ordering an omelette .
label: 1
premise: A person on a horse jumps over a broken down airplane .
hypothesis: A person is outdoors , on a horse .
label: 0



トレーニング セットには約 550,000 ペアがあり、テスト セットには約 10,000 ペアがあります。以下は、トレーニング セットとテスト セットの両方で 3 つの [**ラベル「含意」、「矛盾」、および「中立」がバランスが取れている**] ことを示しています。


In [4]:
test_data = read_snli(data_dir, is_train=False)
for data in [train_data, test_data]:
    print([[row for row in data[2]].count(i) for i in range(3)])

[183416, 183187, 182764]
[3368, 3237, 3219]



### [**データセットをロードするためのクラスの定義**]

以下では、Gluon の`Dataset`クラスを継承して、SNLI データセットをロードするためのクラスを定義します。クラス コンストラクターの引数`num_steps` 、シーケンスの各ミニバッチが同じ形状になるようにテキスト シーケンスの長さを指定します。言い換えると、長いシーケンスの最初の`num_steps`個以降のトークンは削除されますが、特殊なトークン「&amp;lt;pad&amp;gt;」は削除されます。長さが`num_steps`になるまで、短いシーケンスに追加されます。 `__getitem__`関数を実装すると、インデックス`idx`を持つ前提、仮説、ラベルに任意にアクセスできます。


In [5]:
#@save
class SNLIDataset(torch.utils.data.Dataset):
    """A customized dataset to load the SNLI dataset."""
    def __init__(self, dataset, num_steps, vocab=None):
        self.num_steps = num_steps
        all_premise_tokens = d2l.tokenize(dataset[0])
        all_hypothesis_tokens = d2l.tokenize(dataset[1])
        if vocab is None:
            self.vocab = d2l.Vocab(all_premise_tokens + all_hypothesis_tokens,
                                   min_freq=5, reserved_tokens=['<pad>'])
        else:
            self.vocab = vocab
        self.premises = self._pad(all_premise_tokens)
        self.hypotheses = self._pad(all_hypothesis_tokens)
        self.labels = torch.tensor(dataset[2])
        print('read ' + str(len(self.premises)) + ' examples')

    def _pad(self, lines):
        return torch.tensor([d2l.truncate_pad(
            self.vocab[line], self.num_steps, self.vocab['<pad>'])
                         for line in lines])

    def __getitem__(self, idx):
        return (self.premises[idx], self.hypotheses[idx]), self.labels[idx]

    def __len__(self):
        return len(self.premises)


### [**すべてをまとめる**]

これで、 `read_snli`関数と`SNLIDataset`クラスを呼び出して SNLI データセットをダウンロードし、トレーニング セットとテスト セットの両方の`DataLoader`インスタンスをトレーニング セットの語彙とともに返すことができます。トレーニング セットから構築された語彙をテスト セットの語彙として使用する必要があることに注意してください。その結果、テスト セットからの新しいトークンは、トレーニング セットでトレーニングされたモデルには認識されません。


In [6]:
#@save
def load_data_snli(batch_size, num_steps=50):
    """Download the SNLI dataset and return data iterators and vocabulary."""
    num_workers = d2l.get_dataloader_workers()
    data_dir = d2l.download_extract('SNLI')
    train_data = read_snli(data_dir, True)
    test_data = read_snli(data_dir, False)
    train_set = SNLIDataset(train_data, num_steps)
    test_set = SNLIDataset(test_data, num_steps, train_set.vocab)
    train_iter = torch.utils.data.DataLoader(train_set, batch_size,
                                             shuffle=True,
                                             num_workers=num_workers)
    test_iter = torch.utils.data.DataLoader(test_set, batch_size,
                                            shuffle=False,
                                            num_workers=num_workers)
    return train_iter, test_iter, train_set.vocab


ここでは、バッチ サイズを 128 に、シーケンスの長さを 50 に設定し、 `load_data_snli`関数を呼び出してデータ イテレータとボキャブラリを取得します。次に、語彙サイズを出力します。


In [7]:
train_iter, test_iter, vocab = load_data_snli(128, 50)
len(vocab)

read 549367 examples


read 9824 examples


18678


ここで、最初のミニバッチの形状を出力します。センチメント分析とは対照的に、前提と仮説のペアを表す 2 つの入力`X[0]`と`X[1]`があります。


In [8]:
for X, Y in train_iter:
    print(X[0].shape)
    print(X[1].shape)
    print(Y.shape)
    break

torch.Size([128, 50])
torch.Size([128, 50])
torch.Size([128])



## まとめ
- 自然言語推論では、両方がテキスト シーケンスである前提から仮説を推論できるかどうかを研究します。
- 自然言語推論では、前提と仮説の間の関係には、含意、矛盾、中立が含まれます。
-  Stanford Natural Language Inference (SNLI) コーパスは、自然言語推論の人気のベンチマーク データセットです。

## 演習
1. 機械翻訳は長い間、出力翻訳とグラウンドトゥルース翻訳の間の表面的な $n$-gram マッチングに基づいて評価されてきました。自然言語推論を使用して機械翻訳の結果を評価するための尺度を設計できますか?
1. ハイパーパラメータを変更して語彙サイズを減らすにはどうすればよいでしょうか?



[ディスカッション](https://discuss.d2l.ai/t/1388)
